# Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install dependencies
!pip install e3nn==0.4.4 opt_einsum ase torch_ema prettytable

# Clone MACE
!git clone --depth 1 https://github.com/ACEsuit/mace.git

!pip install mace/
!pip install -U numpy==2.0

In [ ]:
from ase.build import bulk
from ase.optimize import BFGS, LBFGS
from ase.constraints import UnitCellFilter, ExpCellFilter
from mace.calculators import MACECalculator
from ase.io import write, read

/usr/local/lib/python3.12/dist-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


# Random Vector Displacement Function

In [ ]:
from ase.io import write
import numpy as np

def random_probes(atoms, m=12, seed=0):
    rng = np.random.default_rng(seed)
    N = len(atoms)
    V = rng.normal(size=(m, N, 3))
    # norm: sum_i m_i ||v_i||^2 = 1
    norms = np.sqrt((V**2).sum(axis=(1,2)) + 1e-30)
    V /= norms[:, None, None]
    return V

def random_vector_displacement_augmented(base_atoms, teacher_calc, m=12, delta=0.02, seed=0, tag="curv"):
    Vs = random_probes(base_atoms, m=m, seed=seed)
    images = []
    R0 = base_atoms.get_positions()
    for j, v in enumerate(Vs):
        for sgn in (+1, -1):
            a = base_atoms.copy()
            a.calc = teacher_calc
            a.set_positions(R0 + sgn * delta * v, apply_constraint=False)
            E = a.get_potential_energy()
            F = a.get_forces()
            # store labels in EXTXYZ-compatible fields
            a.info["energy"] = float(E)
            a.arrays["forces"] = F
            a.info["origin"] = tag
            a.info["probe_id"] = j
            a.info["sign"] = int(sgn)
            images.append(a)
            print(f"Generated {len(images)} images")

    return images


# Generate using e-v window structures + Vacancy structures

In this part the e-v window structures and vacancy structures (5 * 5 * 5 supercell), which were generated using the energy-volume filter to be in the range of 13-20 A3/atom were used for generating new structures

## W

In [ ]:
from ase.io import read, write
evwindow_st_bcc = read ("bcc_W_evwindow.xyz", index=":")
evwindow_st_fcc = read ("fcc_W_evwindow.xyz", index=":")
vac_structures = read("vacancy_W_relaxed.xyz", index=":")

In [ ]:
from ase.io import read, write
MACE_model_path = "2023-12-03-mace-128-L1_epoch-199.model"
MACE_calc = MACECalculator (MACE_model_path, device='cpu', default_dtype='float32')
all_imgs = []
for k, atoms in enumerate(evwindow_st_bcc):
    print (f"Structures for base atoms {k+1} BCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=4, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(evwindow_st_fcc):
    print (f"Structures for base atoms {k+1} FCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=4, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(vac_structures):
    print (f"Structures for base atoms {k+1} Vacancy")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=4, delta=0.4, seed=100+k, tag=f"W_base{k}")

write("random_disp_aug_of_evwindowbccfcc_m4_delta0.4&of_relaxedvacancy250_speedtest.xyz", all_imgs, format="extxyz", write_results=False)


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Structures for base atoms 1 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Generated 5 images
Generated 6 images
Generated 7 images
Generated 8 images
Structures for base atoms 2 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Generated 5 images
Generated 6 images
Generated 7 images
Generated 8 images
Structures for base atoms 3 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Generated 5 images
Generated 6 images
Generated 7 images
Generated 8 images
Structures for base atoms 4 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Generated 5 images
Generated 6 images
Generated 7 images
Generated 8 images
Structures for base atoms 5 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Generated 5 images
Generated 6

## Mo

In [ ]:
from ase.io import read, write
evwindow_st_bcc = read ("bcc_Mo_evwindow.xyz", index=":")
evwindow_st_fcc = read ("fcc_Mo_evwindow.xyz", index=":")
vac_structures = read("vacancy_Mo_relaxed.xyz", index=":")

In [ ]:
from ase.io import read, write
MACE_model_path = "2023-12-03-mace-128-L1_epoch-199.model"
MACE_calc = MACECalculator (MACE_model_path, device='cpu', default_dtype='float32')
all_imgs = []
for k, atoms in enumerate(evwindow_st_bcc):
    print (f"Structures for base atoms {k+1} BCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(evwindow_st_fcc):
    print (f"Structures for base atoms {k+1} FCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(vac_structures):
    print (f"Structures for base atoms {k+1} Vacancy")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")

write("Mo_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz", all_imgs, format="extxyz", write_results=False)


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Structures for base atoms 1 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 2 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 3 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 4 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 5 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 6 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 7 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 8 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures

## Nb

In [ ]:
from ase.io import read, write
evwindow_st_bcc = read ("bcc_Nb_evwindow.xyz", index=":")
evwindow_st_fcc = read ("fcc_Nb_evwindow.xyz", index=":")
vac_structures = read("vacancy_Nb_relaxed.xyz", index=":")

In [ ]:
from ase.io import read, write
MACE_model_path = "2023-12-03-mace-128-L1_epoch-199.model"
MACE_calc = MACECalculator (MACE_model_path, device='cpu', default_dtype='float32')
all_imgs = []
for k, atoms in enumerate(evwindow_st_bcc):
    print (f"Structures for base atoms {k+1} BCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(evwindow_st_fcc):
    print (f"Structures for base atoms {k+1} FCC")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")
for k, atoms in enumerate(vac_structures):
    print (f"Structures for base atoms {k+1} Vacancy")
    all_imgs += random_vector_displacement_augmented(atoms, MACE_calc, m=2, delta=0.4, seed=100+k, tag=f"W_base{k}")

write("Nb_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz", all_imgs, format="extxyz", write_results=False)


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Structures for base atoms 1 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 2 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 3 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 4 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 5 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 6 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 7 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures for base atoms 8 BCC
Generated 1 images
Generated 2 images
Generated 3 images
Generated 4 images
Structures